<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1fc06b15ef2c0dbe5ff70d3bee1bcaa6d06ec4d67424d6687d4fdddbc852dd76
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-08 15:18:11
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.56 C
-------------------
Today PnL: -2.42 L (-1.53%)
Current PnL: -27.21 L (-17.35%)
CY Booked + Current PnL: -12.92 L (-8.24%)
-------------------
Total profit:  1.82 L
Total loss:  -29.03 L
-------------------
Total Booked + Current PnL: 14.06 L (10.83%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.16%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 96.22 L (61.7%)
Deployed:  1.30 C
Current:  1.56 C
CAGR/XIRR %: 10.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.67,15.14,5.44,21.40,16625.0,40178.0,305604.0,6.83,63.0,X-LC,5.53,14.0,2.42,1.96,22.42,XY25,NTT,FMCG
80,TTKPRESTIG,770.00,-1.02,-21.41,27.37,0.09,21676.0,-21581.0,79196.0,83.78,35.0,M-SC,3.24,253.0,-1.00,0.51,4.05,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.39,4.67,13.90,19.23,21891.0,7030.0,157490.0,22.81,74.0,L-LC,6.58,182.0,0.32,1.01,26.94,XY25,ATH,MINING
49,MASFIN,398.61,-4.13,-1.62,24.06,22.05,23191.0,-1590.0,96390.0,-15.09,51.0,H-SC,6.60,168.0,-0.07,0.62,40.11,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.15,-13.33,15.40,0.02,23299.0,-23267.0,151292.0,130.47,58.0,M-SC,7.79,240.0,-1.00,0.97,31.60,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,4.47,-31.55,182.89,93.63,126386.0,-31858.0,69105.0,-26.73,48.0,H-SC,16.52,143.0,-0.25,0.44,4.47,SR,ATH,CHEMICALS
79,TRIDENT,48.00,2.90,-27.71,77.78,28.51,51871.0,-25564.0,66690.0,-14.44,48.0,M-SC,7.20,226.0,-0.49,0.43,14.12,XR,NTT,TEXTILES
32,HCLTECH,1922.01,1.94,6.93,16.65,24.73,43072.0,16765.0,258689.0,11.49,53.0,X-LC,6.44,13.0,0.39,1.66,23.22,X40,ATH,IT
22,DIXON,18931.72,1.84,-16.90,57.95,31.26,111134.0,-38992.0,191776.0,-61.22,34.0,X-MC,18.44,56.0,-0.35,1.23,2.22,X40N,ATH,IT
44,JCHAC,2282.00,0.75,-34.84,53.45,-0.01,44512.0,-44527.0,83278.0,16943.48,48.0,M-SC,13.90,235.0,-1.00,0.53,6.59,OX40N,BTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-4.75,-7.39,57.19,45.57,138560.0,-19339.0,242280.0,-40.52,41.0,H-MC,4.65,121.0,-0.14,1.55,3.09,AR,NTT,ELECTRICAL
68,SIEMENS,4671.50,-4.15,-19.30,55.52,25.51,83383.0,-35910.0,150185.0,-7.73,37.0,H-LC,1.63,50.0,-0.43,0.96,10.67,AR,ATH,ELECTRICAL
49,MASFIN,398.61,-4.13,-1.62,24.06,22.05,23191.0,-1590.0,96390.0,-15.09,51.0,H-SC,6.60,168.0,-0.07,0.62,40.11,XR,ATH,FINANCE
77,TITAGARH,1548.00,-3.77,-22.56,88.18,45.74,165384.0,-54624.0,187553.0,3.16,44.0,H-SC,2.36,175.0,-0.33,1.20,21.98,XY24,NTT,ENGINEERING
56,RAJOOENG,143.10,-3.61,-29.45,97.90,39.61,70791.0,-30190.0,72310.0,-52.38,56.0,H-SC,31.23,136.0,-0.43,0.46,17.04,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-3.44,0.7,19.89,20.74,40695.0,1430.0,204600.0,33.49,59.0,M-MC,5.92,189.0,0.04,1.31,11.39,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.06,-1.94,112.02,107.91,162188.0,-2859.0,144785.0,-20.15,53.0,M-SC,10.19,220.0,-0.02,0.93,7.95,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.15,-13.33,15.40,0.02,23299.0,-23267.0,151292.0,130.47,58.0,M-SC,7.79,240.0,-1.00,0.97,31.60,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,-2.30,-17.58,60.20,32.03,65409.0,-23183.0,108652.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.70,21.70,OX40N,NTT,IT
80,TTKPRESTIG,770.00,-1.02,-21.41,27.37,0.09,21676.0,-21581.0,79196.0,83.78,35.0,M-SC,3.24,253.0,-1.00,0.51,4.05,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.52,-21.82,45.14,13.47,95166.0,-58842.0,210825.0,-67.92,49.0,H-SC,4.80,159.0,-0.62,1.35,7.20,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-3.44,0.70,19.89,20.74,40695.0,1430.0,204600.0,33.49,59.0,M-MC,5.92,189.0,0.04,1.31,11.39,XY25,NTT,FINANCE
17,COALINDIA,484.83,-1.39,4.67,13.90,19.23,21891.0,7030.0,157490.0,22.81,74.0,L-LC,6.58,182.0,0.32,1.01,26.94,XY25,ATH,MINING
87,WIPRO,420.00,-3.25,7.46,60.31,72.27,109818.0,12642.0,182090.0,-7.89,50.0,M-LC,6.13,101.0,0.12,1.17,14.10,XR,NTT,IT
37,IEX,219.00,-3.16,1.72,46.11,48.63,93788.0,3447.0,203401.0,-32.21,62.0,H-SC,13.60,137.0,0.04,1.30,13.54,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-3.25,7.46,60.31,72.27,109818.0,12642.0,182090.0,-7.89,50.0,M-LC,6.13,101.0,0.12,1.17,14.10,XR,NTT,IT
37,IEX,219.00,-3.16,1.72,46.11,48.63,93788.0,3447.0,203401.0,-32.21,62.0,H-SC,13.60,137.0,0.04,1.30,13.54,XR,NTT,MISC
25,FINCABLES,1641.55,-1.06,-1.94,112.02,107.91,162188.0,-2859.0,144785.0,-20.15,53.0,M-SC,10.19,220.0,-0.02,0.93,7.95,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.19,-6.79,121.78,106.72,140001.0,-8374.0,114962.0,-53.87,38.0,H-SC,7.21,138.0,-0.06,0.74,15.00,AR,ATH,MISC
49,MASFIN,398.61,-4.13,-1.62,24.06,22.05,23191.0,-1590.0,96390.0,-15.09,51.0,H-SC,6.60,168.0,-0.07,0.62,40.11,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.35,-16.36,32.45,10.78,65223.0,-39322.0,200996.0,-50.14,13.0,X-LC,5.01,1.0,-0.60,1.29,0.0,X40,NTT,FMCG
7,AWL,485.00,-0.59,-26.21,113.02,57.19,271992.0,-85469.0,240658.0,-65.16,27.0,X-MC,8.81,58.0,-0.31,1.54,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.08,-1.55,-11.84,48.16,30.62,100645.0,-28074.0,208980.0,-37.34,29.0,X-MC,16.17,57.0,-0.28,1.34,0.0,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.01,-40.23,126.23,35.20,97071.0,-51770.0,76900.0,0.79,32.0,X-SC,17.33,93.0,-0.53,0.49,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-1.59,-22.40,82.37,41.51,168331.0,-59005.0,204360.0,-7.63,32.0,X-MC,8.97,65.0,-0.35,1.31,0.0,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,-2.02,-0.83,21.78,20.77,50573.0,-1944.0,232200.0,-9.42,34.0,X-MC,0.98,70.0,-0.04,1.49,6.03,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,-4.15,-19.30,55.52,25.51,83383.0,-35910.0,150185.0,-7.73,37.0,H-LC,1.63,50.0,-0.43,0.96,10.67,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.30,1.54,41.52,43.70,104583.0,3817.0,251885.0,-6.86,64.0,X-MC,2.09,73.0,0.04,1.62,16.82,XY24,BTT,FMCG
45,JIOFIN,387.00,-3.28,-5.58,31.83,24.48,80356.0,-14912.0,252453.0,-7.18,44.0,X-LC,2.19,37.0,-0.19,1.62,46.33,XY24,BTT,FINANCE
84,VBL,671.64,-1.80,1.10,34.19,35.67,109175.0,3477.0,319319.0,-10.92,62.0,X-LC,2.19,5.0,0.03,2.05,15.11,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.74,-27.51,126.46,64.15,218118.0,-65471.0,172480.0,-57.62,37.0,X-SC,6.15,84.0,-0.30,1.11,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-0.59,-26.21,113.02,57.19,271992.0,-85469.0,240658.0,-65.16,27.0,X-MC,8.81,58.0,-0.31,1.54,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.01,-40.23,126.23,35.20,97071.0,-51770.0,76900.0,0.79,32.0,X-SC,17.33,93.0,-0.53,0.49,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-1.59,-22.40,82.37,41.51,168331.0,-59005.0,204360.0,-7.63,32.0,X-MC,8.97,65.0,-0.35,1.31,0.0,XY25,ATH,FMCG
52,PAGEIND,51605.08,-1.55,-11.84,48.16,30.62,100645.0,-28074.0,208980.0,-37.34,29.0,X-MC,16.17,57.0,-0.28,1.34,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.66,-26.77,95.05,42.84,45250.0,-17400.0,47606.0,-52.07,54.0,X-SC,27.86,83.0,-0.38,0.31,7.70,XY24,NTT,MISC
58,RELAXO,1176.00,-0.57,-48.25,194.29,52.31,145960.0,-70035.0,75125.0,-44.92,41.0,X-SC,7.74,92.0,-0.48,0.48,0.56,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.01,-40.23,126.23,35.20,97071.0,-51770.0,76900.0,0.79,32.0,X-SC,17.33,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.08,0.22,23.80,24.08,30281.0,282.0,127232.0,-7.89,51.0,X-SC,6.48,91.0,0.01,0.82,19.88,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-1.10,-17.17,73.04,43.32,98531.0,-27972.0,134900.0,-28.80,46.0,X-SC,7.82,90.0,-0.28,0.87,3.58,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.35,-16.36,32.45,10.78,65223.0,-39322.0,200996.0,-50.14,13.0,X-LC,5.01,1.0,-0.60,1.29,0.00,X40,NTT,FMCG
78,TMPV,600.00,-1.16,-26.39,67.06,22.98,111511.0,-59612.0,166286.0,-23.08,46.0,X-LC,4.50,2.0,-0.53,1.07,4.59,XY24,NTT,AUTO
76,TCS,4389.97,-2.77,-11.90,37.01,20.71,123331.0,-45000.0,333237.0,-24.27,47.0,X-LC,2.59,3.0,-0.36,2.14,11.35,X40,ATH,IT
84,VBL,671.64,-1.80,1.10,34.19,35.67,109175.0,3477.0,319319.0,-10.92,62.0,X-LC,2.19,5.0,0.03,2.05,15.11,X40N,ATH,FMCG
41,INFY,1972.00,-1.62,9.77,22.29,34.24,76558.0,30574.0,343462.0,-14.57,50.0,X-LC,7.61,6.0,0.40,2.20,18.86,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.84,-34.94,104.24,32.89,52364.0,-26972.0,50234.0,-720.45,57.0,L-MC,7.61,259.0,-0.52,0.32,38.34,XR,NTT,BANKS
49,MASFIN,398.61,-4.13,-1.62,24.06,22.05,23191.0,-1590.0,96390.0,-15.09,51.0,H-SC,6.60,168.0,-0.07,0.62,40.11,XR,ATH,FINANCE
13,BSOFT,831.70,-2.84,-21.90,93.06,50.79,102230.0,-30799.0,109854.0,0.12,52.0,H-SC,10.70,171.0,-0.30,0.70,27.91,XR,ATH,IT
72,SURYODAY,216.00,0.19,-17.70,49.49,23.03,72938.0,-31691.0,147380.0,60.09,58.0,H-SC,5.07,165.0,-0.43,0.95,46.23,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.15,-13.33,15.40,0.02,23299.0,-23267.0,151292.0,130.47,58.0,M-SC,7.79,240.0,-1.00,0.97,31.60,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-2.73,-28.75,58.45,12.90,91000.0,-62808.0,155688.0,129.57,64.0,H-SC,10.52,173.0,-0.69,1.00,29.45,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,-2.30,-17.58,60.20,32.03,65409.0,-23183.0,108652.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.70,21.70,OX40N,NTT,IT
17,COALINDIA,484.83,-1.39,4.67,13.90,19.23,21891.0,7030.0,157490.0,22.81,74.0,L-LC,6.58,182.0,0.32,1.01,26.94,XY25,ATH,MINING
85,VOLTAS,1856.00,-2.61,14.87,26.40,45.20,58145.0,28503.0,220245.0,3.62,63.0,X-MC,4.57,80.0,0.49,1.41,22.67,XY25,NTT,AC
36,ICICIPRULI,790.00,-0.51,12.71,15.99,30.73,35504.0,25034.0,222039.0,-13.70,69.0,X-MC,6.47,79.0,0.71,1.42,27.10,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.73
1,25,44.75
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.28
LC    32.63
MC    28.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     24.15
X40      21.81
XY25     18.25
X40N     11.75
XR        8.30
AR        7.81
OX40N     7.03
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    29.49
X-MC    22.24
H-SC    21.00
M-SC    10.36
X-SC     7.20
H-MC     4.21
M-MC     1.31
M-LC     1.17
L-LC     1.01
H-LC     0.96
L-SC     0.72
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.59,-8.40,43.97
IT,11.72,-17.02,76.04
FINANCE,10.18,-12.34,59.48
REFINERIES,9.72,-5.98,32.79
MISC,6.60,-26.33,78.00
ELECTRICAL,5.37,-11.98,53.00
PAINTS,5.24,-12.94,29.72
INSURANCE,4.32,-0.67,35.12
PHARMA,3.61,-4.31,37.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3206173.0,21
AR,1352243.0,10
XR,1198381.0,12
X40,1117773.0,15
X40N,912484.0,9
XY25,856870.0,10
OX40N,696592.0,10
SR,281504.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3518757.0,24
X-MC,1564163.0,16
M-SC,1443420.0,15
X-LC,1387601.0,16
X-SC,799845.0,8
H-MC,421725.0,3
L-SC,178358.0,2
M-LC,109818.0,1
H-LC,83383.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247103.0      6
           AR         917048.0      5
M-SC       XY24       832384.0      6
H-SC       XR         768346.0      7
X-LC       XY25       567808.0      6
X-MC       X40        501349.0      7
           XY24       441552.0      3
X-LC       X40        422528.0      6
X-MC       X40N       394786.0      4
X-SC       X40N       312300.0      3
H-SC       OX40N      304756.0      4
M-SC       OX40N      300013.0      5
X-SC       XY24       293649.0      3
H-SC       SR         281504.0      2
X-MC       XY25       226476.0      2
H-MC       AR         222107.0      2
X-LC       X40N       205398.0      2
H-MC       XY24       199618.0      1
X-SC       X40        193896.0      2
X-LC       XY24       191867.0      2
M-SC       XR         181318.0      2
           AR         129705.0      2
M-LC       XR         109818.0      1
L-SC       OX40N       91823.0      1
           XR          86535.0      1
H-LC       AR          83383.0      1
L-MC       XR          52364.0      1
M-MC       XY25        40695.0      1
L-LC       XY25        21891.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
